## Подготовка

---

---

### Загрузим датасет, познакомимся с данными

---

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm.notebook import tqdm
from tqdm import notebook

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

import torch
import transformers as ppb
from transformers import AutoModel, AutoTokenizer 
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/admin_mbp_15_2012/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/admin_mbp_15_2012/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


**Объявим константу:**

In [2]:
RS = 123

**Загрузим датасет**

In [3]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [4]:
df.head(3)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0


---

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


---

**Промежуточный вывод:**

**Данные в таблице представлены двумя типами данных:**
* object
* int64

**Столбец `text` содержит текст комментария, а столбец `toxic` — целевой признак.**

**В данных нету пропусков.**

**В названии колонкок нету нарушения стиля записи.**

**Данные в колонке 'Unnamed: 0' повторяют номера индексов, эта колонка с неинформативным признаком, удалим ее в дальнейшем:**

---

### Неинформативные признаки

---

**Удалим колонку 'Unnamed: 0' с неиформативным признаком:**

In [6]:
df = df.drop('Unnamed: 0', axis=1)

**Проверим результат:**

In [7]:
df.head(3)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


---

### Дубликаты

---

**Проверим есть ли в данных явные дубликаты:**

In [8]:
df.duplicated().sum()

0

**Явных дубликатов не обнаружили.**

---

### Деление на выборки

---

**Выделим признак и целевой признак:**

In [9]:
features = df.drop('toxic', axis=1)
target = df['toxic']

**Поделим данные на выборки:**

In [10]:
features_train, features_test, target_train, target_test = train_test_split(
features, target, test_size=0.25, stratify=target, random_state=RS)

In [11]:
features_train.reset_index(drop= True , inplace= True )
features_test.reset_index(drop= True , inplace= True )
target_train.reset_index(drop= True , inplace= True )
target_test.reset_index(drop= True , inplace= True )

---

### Лемматизация текстов

---

**Создадим функцию для лемматизации текстов:**

In [12]:
def lem_text(text):
    
    final = []
    for t in notebook.tqdm(text):
        t = " ".join(re.sub(r'[^a-zA-Z]', ' ', t).split())
        token_list = word_tokenize(t)
        
        stop_text = []
        for word in token_list:
            if word.lower() not in stopwords.words('english'):
                stop_text.append(word.lower())
        
        lemmatizer = WordNetLemmatizer()
        lemm_list = []
        for w in stop_text:
            lem_t = lemmatizer.lemmatize(w)
            lemm_list.append(lem_t)
        lemm_list = " ".join(lemm_list)
        final.append(lemm_list)
        
    return final

**Применим функцию, к новому столбцу, который создадим в тренировочной выборке:**

In [13]:
features_train['lemm_text'] = lem_text(features_train['text'])

  0%|          | 0/119469 [00:00<?, ?it/s]

**Лемматизация длилась 27 минут, посмотрим результат:**

In [14]:
features_train.head(3)

,text,lemm_text
0,SA article \n\nI'm not just blanking whatever ...,sa article blanking whatever want removing eve...
1,""" \n\n Timeline February 8, 2005: Essjay accou...",timeline february essjay account registered es...
2,"""\nVandalism was added with the promorion of s...",vandalism added promorion show people initial ...


**Применим функцию, к новому столбцу, который создадим в тестовой выборке:**

In [15]:
%%time
features_test['lemm_text'] = lem_text(features_test['text'])

  0%|          | 0/39823 [00:00<?, ?it/s]

CPU times: user 6min 21s, sys: 1min 53s, total: 8min 15s
Wall time: 8min 14s


**Лемматизация длилась 9 минут, посмотрим результат:**

In [16]:
features_test.head(3)

,text,lemm_text
0,Give me an email address where I may email you...,give email address may email concern please
1,"""\n\n Genetics \n\nThis recent edit added to t...",genetics recent edit added genetics section tw...
2,2012 (UTC)\n You expected more from a liberal ...,utc expected liberal rag like wakopedia racist...


---

### Корпусы текстов

---

**Создадим корпусы текстов для обучающей и тестовой выборок:**

In [17]:
corpus_train = list(features_train['lemm_text'].values)
corpus_test = list(features_test['lemm_text'].values)

---

### TF-IDF

---

**Вычислим TF-IDF для корпусов тренировочной и тестовой выборок:**

In [18]:
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/admin_mbp_15_2012/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
count_tf_idf_train = TfidfVectorizer(stop_words=stopwords)

In [20]:
tf_idf_train = count_tf_idf_train.fit_transform(corpus_train)
tf_idf_test = count_tf_idf_train.transform(corpus_test)

---

**Промежуточный вывод**

**В ходе подготовки мы выполнили такие шаги:**
- познакомились с датасетом
- удалили неинформативный признак
- выяснили, что нету пропусков
- выяснили, что нет дубликатов
- сделали лемматизацию текстов
- подготовили корпусы текстов

---

---

## Обучение

---

---

**С помощью ранее созданного корпуса обучим разные модели:**

### Обучим модель логистической регрессии

In [21]:
pipe_lr = Pipeline([('tfidf', TfidfVectorizer()),
                ('clf', LogisticRegression(random_state=RS))])
 
scores = cross_val_score(pipe_lr, corpus_train, target_train,
                                     scoring = 'f1', cv=3, n_jobs=-1).mean()
 
model_lr = pipe_lr.fit(corpus_train, target_train)
print("F1 мера =", round(scores))

/Users/admin_mbp_15_2012/opt/anaconda3/envs/ds_practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


F1 мера = 1


---

### Обучим модель случайного леса

---

In [22]:
def rfr_f1(x_train, y_train):
    model = Pipeline([('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier(random_state=RS))])
    
    parametrs = {'clf__n_estimators': range (1, 15, 3),
                 'clf__max_depth': range (1, 15, 3), 
                 'clf__min_samples_split': range (2, 9, 2)}
    
    model_rfr = GridSearchCV(model, parametrs, cv=3, scoring = 'f1', n_jobs=-1)
    model_rfr.fit(x_train, y_train)
    return model_rfr

In [23]:
%%time
model_rfr = rfr_f1(corpus_train, target_train)
print('Лучшие параметры модели =', model_rfr.best_params_, 'F1 =', round(model_rfr.best_score_))

Лучшие параметры модели = {'clf__max_depth': 13, 'clf__min_samples_split': 6, 'clf__n_estimators': 1} F1 = 0
CPU times: user 24 s, sys: 14.3 s, total: 38.4 s
Wall time: 6min 41s


---

### Обучим модель решающего дерева

---

In [24]:
def tree_f1(x_train, y_train):
    model = Pipeline([('tfidf', TfidfVectorizer()),
                 ('clf', DecisionTreeClassifier(random_state=RS))])
    
    parametrs = {'clf__max_depth': range (1, 15, 3), 
                 'clf__min_samples_split': range (2, 9, 2)}
    
    model_tree = GridSearchCV(model, parametrs, cv=3, scoring = 'f1', n_jobs=-1)
    model_tree.fit(x_train, y_train)
    return model_tree

In [25]:
%%time
model_tree = tree_f1(corpus_train, target_train)
print('Лучшие параметры модели =', model_tree.best_params_, 'F1 =', round(model_tree.best_score_))

Лучшие параметры модели = {'clf__max_depth': 13, 'clf__min_samples_split': 6} F1 = 1
CPU times: user 14.9 s, sys: 2.87 s, total: 17.8 s
Wall time: 2min 3s


---

### Обучим модель CatBoostClassifier

---

In [26]:
def catboost(x_train, y_train):
    cat = CatBoostClassifier(random_state=RS,
                            loss_function='Logloss',
                            early_stopping_rounds=5,
                            iterations=300,
                            verbose=100)
    
    pipe_cat = Pipeline([('tfidf', TfidfVectorizer()), 
                         ('clf', cat)])
        
    scores_cat = cross_val_score(pipe_cat, x_train, y_train,
                                     scoring = 'f1', cv=3, n_jobs=-1).mean()
    
    pipe_cat.fit(x_train, y_train)
    print("F1 мера =", scores_cat)
    return pipe_cat

In [27]:
%%time
cat_model =  catboost(corpus_train, target_train)

Learning rate set to 0.20147
0:	learn: 0.5101545	total: 3.9s	remaining: 19m 25s
100:	learn: 0.1408232	total: 2m 19s	remaining: 4m 35s
200:	learn: 0.1203324	total: 4m 33s	remaining: 2m 14s
299:	learn: 0.1085701	total: 6m 44s	remaining: 0us
Learning rate set to 0.20147
0:	learn: 0.5093267	total: 4.02s	remaining: 20m 2s
100:	learn: 0.1403395	total: 2m 20s	remaining: 4m 35s
200:	learn: 0.1203905	total: 4m 34s	remaining: 2m 15s
299:	learn: 0.1083183	total: 6m 45s	remaining: 0us
Learning rate set to 0.20147
0:	learn: 0.5158732	total: 3.96s	remaining: 19m 42s
100:	learn: 0.1412694	total: 2m 20s	remaining: 4m 36s
200:	learn: 0.1202942	total: 4m 34s	remaining: 2m 15s
299:	learn: 0.1081968	total: 6m 45s	remaining: 0us
Learning rate set to 0.239553
0:	learn: 0.4720626	total: 861ms	remaining: 4m 17s
100:	learn: 0.1373357	total: 1m 12s	remaining: 2m 23s
200:	learn: 0.1179734	total: 2m 24s	remaining: 1m 11s
299:	learn: 0.1075454	total: 3m 35s	remaining: 0us
F1 мера = 0.7445378965577891
CPU times: us

**На кросс-валидации получили F1-меру = 0.74**

---

### BERT

---

**Загрузка предобученной модели DistilBERT и загрузка токенизатора:**

In [28]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel,
                                                    ppb.DistilBertTokenizer,
                                                    'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**Выберем небольшую часть датасета случайным образом:**

In [29]:
df_bert = df.copy()
df_bert = df_bert.sample(1000).reset_index(drop=True)

---

**Выполним токенизацию:**

In [30]:
%%time
tokenized = df_bert['text'].apply(
    (lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=150))
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


CPU times: user 2.28 s, sys: 21.2 ms, total: 2.3 s
Wall time: 2.31 s


---

**Приведем векторы к одному размеру путем добавления нулей до длинны максимально длинного вектора:**

In [31]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [32]:
np.array(padded).shape

(1000, 150)

---

**Создадим маску, чтобы модель в дальнейшем не учитывала нули, которыми мы уравняли векторы в предыдущем пункте:**

In [33]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 150)

---

**Создание входного вектора и передача его в модель DistilBERT:**

In [34]:
%%time
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

CPU times: user 5min 52s, sys: 36.7 s, total: 6min 28s
Wall time: 5min 43s


---

**Получим массив из эмбеддингов в качестве признаков:**

In [35]:
bert_features = last_hidden_states[0][:,0,:].numpy()

---

**Отделим целевой признак и получим выборки:**

In [36]:
bert_target = df_bert['toxic']

In [37]:
bert_train_features, bert_test_features, bert_train_target, bert_test_target = train_test_split(
    bert_features,
    bert_target,
    test_size=0.5,
    random_state=RS
)

---

**Обучим логистическую регресию на полученных выборках и найдем F1-меру:**

In [38]:
lr_bert = LogisticRegression()
lr_bert.fit(bert_train_features, bert_train_target)

LogisticRegression()

In [39]:
pred_bert = lr_bert.predict(bert_test_features)
f1_bert = f1_score(pred_bert, bert_test_target)
print('F1 мера =', round(f1_bert))

F1 мера = 1


---

**Промежуточный вывод:**

**Провели обучение и сравнили F1-меру таких моделей:**
- LogisticRegression
- DecisionTreeClassifier
- RandomForestClassifier
- CatBoostClassifier

**По результатам сравнения F1-меры, лучшей моделью будем считать - CatBoostClassifier.**

---

---

### TOXIC-BERT

**Загрузка предобученной модели TOXIC-BERT и загрузка токенизатора:**

In [40]:
tokenizer_toxic = AutoTokenizer.from_pretrained("unitary/toxic-bert")
model_toxic = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

In [41]:
model_class_t, tokenizer_class_t, pretrained_weights_t = (ppb.BertModel,
                                                    ppb.BertTokenizer,
                                                    'unitary/toxic-bert')

tokenizer_toxic = tokenizer_class_t.from_pretrained(pretrained_weights_t)
model_toxic = model_class_t.from_pretrained(pretrained_weights_t)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


---

**Выберем небольшую часть датасета случайным образом:**

In [42]:
df_bert_toxic = df.copy()
df_bert_toxic = df_bert_toxic.sample(300).reset_index(drop=True)

---

**Выполним токенизацию:**

In [43]:
%%time
tokenized_toxic = df_bert_toxic['text'].apply(
    (lambda x: tokenizer_toxic.encode(x, add_special_tokens=True, max_length=50))
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


CPU times: user 682 ms, sys: 9.4 ms, total: 691 ms
Wall time: 691 ms


---

**Приведем векторы к одному размеру путем добавления нулей до длинны максимально длинного вектора:**

In [44]:
max_len = 0
for i in tokenized_toxic.values:
    if len(i) > max_len:
        max_len = len(i)

padded_toxic = np.array([i + [0]*(max_len-len(i)) for i in tokenized_toxic.values])

**Проверим размер:**

In [45]:
np.array(padded_toxic).shape

(300, 50)

---

**Создадим маску, чтобы модель в дальнейшем не учитывала нули, которыми мы уравняли векторы в предыдущем пункте:**

In [46]:
attention_mask_toxic = np.where(padded_toxic != 0, 1, 0)
attention_mask_toxic.shape

(300, 50)

---

**Создание входного вектора и передача его в модель TOXIC-BERT:**

In [47]:
%%time
input_ids_toxic = torch.tensor(padded_toxic)  
attention_mask_toxic = torch.tensor(attention_mask_toxic)

with torch.no_grad():
    last_hidden_states_toxic = model_toxic(input_ids_toxic, attention_mask=attention_mask_toxic)

CPU times: user 1min 7s, sys: 3.23 s, total: 1min 10s
Wall time: 1min 4s


---

**Получим массив из эмбеддингов в качестве признаков:**

In [48]:
bert_features_toxic = last_hidden_states_toxic[0][:,0,:].numpy()

---

**Отделим целевой признак и получим выборки:**

In [49]:
bert_target_toxic = df_bert_toxic['toxic']

In [50]:
bert_train_features_t, bert_test_features_t, bert_train_target_t, bert_test_target_t = train_test_split(
    bert_features_toxic,
    bert_target_toxic,
    test_size=0.5,
    random_state=RS
)

---

**Обучим логистическую регресию на полученных выборках и найдем F1-меру:**

In [51]:
lr_bert_tox = LogisticRegression()
lr_bert_tox.fit(bert_train_features_t, bert_train_target_t)

LogisticRegression()

In [52]:
pred_bert_toxic = lr_bert_tox.predict(bert_test_features_t)
f1_bert_toxic = f1_score(pred_bert_toxic, bert_test_target_t)
print('F1 мера =', round(f1_bert_toxic))

F1 мера = 1


## Тестирование

---

---

**Проведем тестирование, найденной лучшей модели на тестовой выбоке:**

In [53]:
pred_cat_test = cat_model.predict(corpus_test)
cat_f1_test = f1_score(pred_cat_test, target_test)
print('F1 модели =', cat_f1_test)

F1 модели = 0.7623563218390805


**Промежуточный вывод:**

**В результате проверки модели CatBoostClassifier на тестовых данных - получаем F1-меру 0.75, такой результат нас устраивает.**

---

---

## Общий вывод

---

**Мы провели исследование с целью создать инструмент для интернет-магазина «Викишоп», который будет искать токсичные комментарии.**

**В ходе исследования мы проделали такие шаги:**
- после подготовки получили очищенные и лемматизированные данные
- создали TF-IDF для обучения моделей
- обучили 3 модели, включая 1 модель с градиентным бустингом
- провели проверку лучшей модели на тестовых данных

**По результатам проверки лучшей модели, а именно CatBoostClassifier, на тестовых данных, мы получили F1-меру = 0.75, рекомендуется использовать именно эту модель для поиска токсичных комментариев.**